<h2>🔍 FreshCart Churn Prediction - Model Evaluation & Interpretation</h2>
 
<h4>
    <b>
        Zero2End Machine Learning Bootcamp - Final Project
    </b>
</h4>

<h3>
    📋 Notebook Contents
</h3>

<ol>
    <li>Load Final Model</li>
    <li>Confusion Matrix & Error Analysis</li>
    <li>Feature Importance Analysis</li>
    <li>SHAP Values & Interpretation</li>
    <li>Partial Dependence Plots</li>
    <li>Business Metrics & Validation</li>
    <li>Model Decision Analysis</li>
</ol>

<hr>

In [4]:
# Library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import sys
import json

import lightgbm as lgb
import shap
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve,
    roc_auc_score, average_precision_score
)
import joblib

In [5]:
# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [6]:
# Add src to path
sys.path.append('../src')

In [7]:
from config import PROCESSED_DATA_DIR, MODEL_DIR, RANDOM_STATE, BUSINESS_METRICS

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


<h4>
    1️⃣ Load Final Model and Data
</h4>

In [8]:
# Load final model
print("📦 Loading final model and data...")

final_model = joblib.load(MODEL_DIR / 'final_model_optimized.pkl')

with open(MODEL_DIR / 'final_metrics.json', 'r') as f:
    final_metrics = json.load(f)

with open(MODEL_DIR / 'feature_names.json', 'r') as f:
    feature_names = json.load(f)

📦 Loading final model and data...


In [10]:
# Load data
final_features = pd.read_parquet(PROCESSED_DATA_DIR / 'final_features_advanced.parquet')

X = final_features[feature_names].fillna(0)
y = final_features['is_churn']

In [11]:
# Split (same as training)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"✅ Model and data loaded")
print(f"   Test set: {X_test.shape}")
print(f"   Model performance (F1): {final_metrics['f1']:.4f}")

✅ Model and data loaded
   Test set: (41242, 53)
   Model performance (F1): 1.0000


In [ ]:
# Get predictions
y_pred = (final_model.predict(X_test) > 0.5).astype(int)
y_pred_proba = final_model.predict(X_test)

print("\n📊 Prediction distribution:")
print(f"   Predicted Churned: {y_pred.sum():,} ({y_pred.mean():.2%})")
print(f"   Predicted Active:  {(y_pred == 0).sum():,} ({(y_pred == 0).mean():.2%})")
print(f"   Actual Churned:    {y_test.sum():,} ({y_test.mean():.2%})")


<h4>
    2️⃣ Confusion Matrix & Error Analysis
</h4>

In [12]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Confusion matrix (counts)
ax = axes[0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
            xticklabels=['Active', 'Churned'],
            yticklabels=['Active', 'Churned'])
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
ax.set_title('Confusion Matrix (Counts)')

# Confusion matrix (percentages)
ax = axes[1]
cm_pct = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
sns.heatmap(cm_pct, annot=True, fmt='.1f', cmap='Blues', ax=ax,
            xticklabels=['Active', 'Churned'],
            yticklabels=['Active', 'Churned'])
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
ax.set_title('Confusion Matrix (Percentages)')

plt.tight_layout()
plt.savefig('../plots/12_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

NameError: name 'y_pred' is not defined